In [95]:
#On dispose d'une image d'entrée carrée en niveau de gris
from PIL import Image
import numpy as np

def image_to_gray_matrix(image_path):
    """
    Convertit une image en niveaux de gris en une matrice de niveaux de gris.

    :param image_path: Chemin de l'image à convertir.
    :return: Matrice numpy représentant les niveaux de gris de l'image.
    """
    # Charger l'image
    image = Image.open(image_path)

    # Convertir l'image en niveaux de gris si ce n'est pas déjà le cas
    if image.mode != 'L':
        image = image.convert('L')

    # Convertir l'image en matrice numpy
    image_matrix = np.array(image)

    return image_matrix


#On répète jusqu'a ce que la distortion moyenne devienne inferieure a un certain seuil
#A la fin de l'étape précédente on obtient ainsi deux vecteurs qui sont les centres de gravité des deux classes. On split ces deux vecteurs selon la ligne 5 encore, on applique l'étape 3 à nouveau sur ces 2 vecteurs. on s'arrete quand le nombre voulu de vecteurs prototypes (la par exemple on en a 4).
#Une fois le dictionnaire formé, on remplace chaque bloc de l'image par son protoype le plus proche au sens de la distance euclidienne.

In [112]:
#On sépare cette image en bloc de taille NxN
def split_matrix_into_blocks(matrix, block_size):
    """
    Sépare une matrice en blocs de taille NxN.

    :param matrix: Matrice numpy à séparer.
    :param block_size: Taille des blocs NxN.
    :return: Liste de blocs de taille NxN.
    """
    # Obtenir les dimensions de la matrice
    rows, cols = matrix.shape

    # Vérifier que la matrice peut être divisée en blocs de taille NxN
    if rows % block_size != 0 or cols % block_size != 0:
        raise ValueError("La taille de la matrice doit être un multiple de la taille des blocs.")

    # Initialiser la liste de blocs
    blocks = []

    # Séparer la matrice en blocs
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            block = matrix[i:i + block_size, j:j + block_size]
            blocks.append(block)

    return blocks

blocks_array = np.array(blocks)
print(blocks_array.shape)
print(blocks_array)


(262144,)
[207. 207. 207. ...  76.  80.  91.]


In [97]:
#Chaque bloc est mappé en vecteur de dimension N²
def block_to_vector(block):
    """
    Transforme un block de taille N en un vecteur de dimension n² en empilant les colonnes.
    
    """
    # Vérifier que le block est carré
    rows, cols = block.shape
    if rows != cols:
        raise ValueError("La matrice doit être carrée.")

    # Empiler les colonnes de la matrice l'une en dessous de l'autre
    vector = matrix.T.flatten()

    return vector

def matrix_to_vectors(matrix,N):
    matrix_blocks = []
    blocks_list = split_matrix_into_blocks(matrix,N)
    for k in range(len(blocks_list)):
        vector = block_to_vector(blocks_list[k])
        matrix_blocks.append(vector)
    return matrix_blocks

In [98]:
#On détermine le vecteur qui est le centre de gravité c0 de tous les vecteurs
def center_of_gravity(vectors):
    """
    Détermine le centre de gravité d'une liste de vecteurs.

    :param vectors: Liste de vecteurs numpy.
    :return: Vecteur numpy représentant le centre de gravité.
    """
    # Vérifier que la liste de vecteurs n'est pas vide
    if len(vectors) == 0:
        raise ValueError("La liste de vecteurs ne peut pas être vide.")
    
    # Convertir la liste de vecteurs en une matrice numpy pour faciliter les opérations
    vector_matrix = np.array(vectors)
    
    # Calculer la somme de tous les vecteurs
    sum_vector = np.sum(vector_matrix, axis=0)
    
    # Calculer le centre de gravité en divisant par le nombre de vecteurs
    center_vector = sum_vector / len(vectors)
    
    return center_vector

In [99]:
#On split ce vecteur en 2 vecteurs c01=c0+eps et c11=c0-eps
def split_vector(vector,eps):
    return [vector+eps, vector-eps]

In [100]:
#On classe tous les autres vecteurs de la base d'apprentissage relativement a ces 2 vecteurs en utilisant le critère de la distance euclienne minimale.
def class_vector(vector, classes):
    class_v = 0
    dist_min = distance_euclienne(vector, classes[0])
    for k in range(1, len(classes)):
        if distance_euclienne(vector,classes[k])<dist_min:
            dist_min = distance_euclienne(vector, classes[k])
            class_v = k
    return class_v
        
def distance_euclienne(v1, v2):
    return np.linalg.norm(v1-v2)


In [101]:
#On obtient 2 classes de vecteurs. Pour chaque classe, on calcule le centre de gravité. les deux vecteurs obtenus remplacent c01 et c11.
def distortion_moyenne(clusters, centers):
    total_distortion = 0
    total_vectors = 0

    for cluster, center in zip(clusters, centers):
        cluster = np.array(cluster)
        center = np.array(center)

        # Calcul de la distance entre chaque vecteur et son centre de gravité
        distances = np.linalg.norm(cluster - center, axis=1) ** 2
        total_distortion += np.sum(distances)
        total_vectors += len(cluster)

    mean_distortion = total_distortion / total_vectors
    return mean_distortion

In [102]:
def LBG(image_path, N, delta, n): #N: taille de bloc voulue ; delta: distortion max threshold ; n: nombre voulu de vecteurs prototypes.
    matrix = image_to_gray_matrix(image_path)
    vectors = matrix_to_vectors(matrix,N)    
    eps = np.random.uniform(0.0001, 0.0002, N*N)
    c0 = center_of_gravity(vectors)
    prototypes = [c0]
    while len(prototypes) < n:
        classes = []
        for vector in prototypes:
            classes.append(split_vector(vector,eps)[0])
            classes.append(split_vector(vector,eps)[1])
        mean_distortion_before = 0
        mean_distortion_after = 0.1
        while abs(mean_distortion_after - mean_distortion_before) >= delta:
            clusters = [[] for k in range(len(classes))]
            for vector in vectors:
                clusters[class_vector(vector, classes)].append(vector)
            centers = [[] for k in range(len(classes))]
            for k in range(len(clusters)):
                centers[k].append(center_of_gravity(clusters[k]))
            mean_distortion_before = mean_distortion_after
            mean_distortion_after = distortion_moyenne(clusters, centers)
            prototypes = centers
    for k in range(len(vectors)):
        vectors[k] = prototypes[class_vector(vector, prototypes)]
        

In [107]:
matrix = image_to_gray_matrix("C:/Users/Axel/LBG/test.jpg")
print(matrix.shape)

(512, 512)


In [109]:
print(block)

NameError: name 'block' is not defined

In [108]:
blocks = split_matrix_into_blocks(matrix,16)
print(blocks)

[207. 207. 207. ...  76.  80.  91.]


In [105]:
vectors = matrix_to_vectors(matrix,16)
print(vectors)

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
center = center_of_gravity(vectors)
print(center)

In [106]:
LBG("C:/Users/Axel/LBG/test.jpg",16,0.0001,256)

ValueError: not enough values to unpack (expected 2, got 0)